In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
td=pd.read_csv('/kaggle/input/new-york-city-taxi-fare-prediction/train.csv',nrows=10_000_000)
td.head()#td:train data # ted:test data

In [ ]:
td.shape

In [ ]:
td.info()

In [ ]:
ted=pd.read_csv('/kaggle/input/new-york-city-taxi-fare-prediction/test.csv')
ted.head()

In [ ]:
ted.info()

In [ ]:
td.isna().sum()

In [ ]:
td['Difference_longitude']=np.abs(np.asarray(td['pickup_longitude']-td['dropoff_longitude']))
td['Difference_latitude']=np.abs(np.asarray(td['pickup_latitude']-td['dropoff_latitude']))


ted['Difference_longitude']=np.abs(np.asarray(ted['pickup_longitude']-ted['dropoff_longitude']))
ted['Difference_latitude']=np.abs(np.asarray(ted['pickup_latitude']-ted['dropoff_latitude']))

In [ ]:
print(f'Before Dropping null values: {len(td)}')
td.dropna(inplace=True)
print(f'After Dropping null values: {len(td)}')

In [ ]:
plot = td[:2000].plot.scatter('Difference_longitude', 'Difference_latitude')

In [ ]:
td=td[(td['Difference_longitude']<5.0)&(td['Difference_latitude']<5.0)]

In [ ]:
l1=list(td['pickup_datetime'])
for i in range(len(l1)):
    l1[i]=l1[i][11:-7:]
td['pickuptime']=l1    



l1=list(ted['pickup_datetime'])
for i in range(len(l1)):
    l1[i]=l1[i][11:-7:]
ted['pickuptime']=l1   

In [ ]:
td.head()

In [ ]:
l1=list(td['pickup_datetime'])
for i in range(len(l1)):
    l1[i]=l1[i][:-4:]
    l1[i]=pd.Timestamp(l1[i])
    l1[i]=l1[i].weekday()
td['Weekday']=l1


l1=list(ted['pickup_datetime'])
for i in range(len(l1)):
    l1[i]=l1[i][:-4:]
    l1[i]=pd.Timestamp(l1[i])
    l1[i]=l1[i].weekday()
ted['Weekday']=l1

td.head()



In [ ]:
ted.head()

In [ ]:
td.drop('pickup_datetime',inplace=True,axis=1)
ted.drop('pickup_datetime',inplace=True,axis=1)

td['Weekday'].replace(to_replace=[i for i in range(0,7)],
                            value=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'],
                              inplace=True)
ted['Weekday'].replace(to_replace=[i for i in range(0,7)],
                              value=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'],
                              inplace=True)

th=pd.get_dummies(td['Weekday'])
teh=pd.get_dummies(ted['Weekday'])
td=pd.concat([td,th],axis=1)
ted=pd.concat([ted,teh],axis=1)

td.drop('Weekday',axis=1,inplace=True)
ted.drop('Weekday',axis=1,inplace=True)

l1=list(td['pickuptime'])
for i in range(len(l1)):
    z=l1[i].split(':')
    l1[i]=int(z[0])*100+int(z[1])
td['pickuptime']=l1


l1=list(ted['pickuptime'])
for i in range(len(l1)):
    z=l1[i].split(':')
    l1[i]=int(z[0])*100+int(z[1])
ted['pickuptime']=l1

td.head()



In [ ]:
R = 6373.0
lat1 =np.asarray(np.radians(td['pickup_latitude']))
lon1 = np.asarray(np.radians(td['pickup_longitude']))
lat2 = np.asarray(np.radians(td['dropoff_latitude']))
lon2 = np.asarray(np.radians(td['dropoff_longitude']))

dlon = lon2 - lon1
dlat = lat2 - lat1
l1=[] 
a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/ 2)**2
c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
distance = R * c

    
td['Distance']=np.asarray(distance)*0.621



lat1 =np.asarray(np.radians(ted['pickup_latitude']))
lon1 = np.asarray(np.radians(ted['pickup_longitude']))
lat2 = np.asarray(np.radians(ted['dropoff_latitude']))
lon2 = np.asarray(np.radians(ted['dropoff_longitude']))

dlon = lon2 - lon1
dlat = lat2 - lat1
 
a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/ 2)**2
c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
distance = R * c
ted['Distance']=np.asarray(distance)*0.621

In [ ]:
R = 6373.0
lat1 =np.asarray(np.radians(td['pickup_latitude']))
lon1 = np.asarray(np.radians(td['pickup_longitude']))
lat2 = np.asarray(np.radians(td['dropoff_latitude']))
lon2 = np.asarray(np.radians(td['dropoff_longitude']))

lat3=np.zeros(len(td))+np.radians(40.6413111)
lon3=np.zeros(len(td))+np.radians(-73.7781391)
dlon_pickup = lon3 - lon1
dlat_pickup = lat3 - lat1
d_lon_dropoff=lon3 -lon2
d_lat_dropoff=lat3-lat2
a1 = np.sin(dlat_pickup/2)**2 + np.cos(lat1) * np.cos(lat3) * np.sin(dlon_pickup/ 2)**2
c1 = 2 * np.arctan2(np.sqrt(a1), np.sqrt(1 - a1))
distance1 = R * c1
td['Pickup_Distance_airport']=np.asarray(distance1)*0.621

a2=np.sin(d_lat_dropoff/2)**2 + np.cos(lat2) * np.cos(lat3) * np.sin(d_lon_dropoff/ 2)**2
c2 = 2 * np.arctan2(np.sqrt(a2), np.sqrt(1 - a2))
distance2 = R * c2

    
td['Dropoff_Distance_airport']=np.asarray(distance2)*0.621



lat1 =np.asarray(np.radians(ted['pickup_latitude']))
lon1 = np.asarray(np.radians(ted['pickup_longitude']))
lat2 = np.asarray(np.radians(ted['dropoff_latitude']))
lon2 = np.asarray(np.radians(ted['dropoff_longitude']))

lat3=np.zeros(len(ted))+np.radians(40.6413111)
lon3=np.zeros(len(ted))+np.radians(-73.7781391)
dlon_pickup = lon3 - lon1
dlat_pickup = lat3 - lat1
d_lon_dropoff=lon3 -lon2
d_lat_dropoff=lat3-lat2
a1 = np.sin(dlat_pickup/2)**2 + np.cos(lat1) * np.cos(lat3) * np.sin(dlon_pickup/ 2)**2
c1 = 2 * np.arctan2(np.sqrt(a1), np.sqrt(1 - a1))
distance1 = R * c1
ted['Pickup_Distance_airport']=np.asarray(distance1)*0.621

a2=np.sin(d_lat_dropoff/2)**2 + np.cos(lat2) * np.cos(lat3) * np.sin(d_lon_dropoff/ 2)**2
c2 = 2 * np.arctan2(np.sqrt(a2), np.sqrt(1 - a2))
distance2 = R * c2

    
ted['Dropoff_Distance_airport']=np.asarray(distance2)*0.621

td['Distance']=np.round(td['Distance'],2)
td['Pickup_Distance_airport']=np.round(td['Pickup_Distance_airport'],2)
td['Dropoff_Distance_airport']=np.round(td['Dropoff_Distance_airport'],2)
ted['Distance']=np.round(ted['Distance'],2)
ted['Pickup_Distance_airport']=np.round(ted['Pickup_Distance_airport'],2)
ted['Dropoff_Distance_airport']=np.round(ted['Dropoff_Distance_airport'],2)

td.drop(['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude'],axis=1,inplace=True)
ted.drop(['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude'],axis=1,inplace=True)

td['Difference_longitude']=np.abs(td['Difference_longitude']-np.mean(td['Difference_longitude']))
td['Difference_longitude']=td['Difference_longitude']/np.var(td['Difference_longitude'])

td['Difference_latitude']=np.abs(td['Difference_latitude']-np.mean(td['Difference_latitude']))
td['Difference_latitude']=td['Difference_latitude']/np.var(td['Difference_latitude'])

ted['Difference_longitude']=np.abs(ted['Difference_longitude']-np.mean(ted['Difference_longitude']))
ted['Difference_longitude']=ted['Difference_longitude']/np.var(ted['Difference_longitude'])

ted['Difference_latitude']=np.abs(ted['Difference_latitude']-np.mean(ted['Difference_latitude']))
ted['Difference_latitude']=ted['Difference_latitude']/np.var(ted['Difference_latitude'])

td.shape



In [ ]:
ted.shape

In [ ]:
from sklearn.model_selection import train_test_split
X=td.drop(['key','fare_amount'],axis=1)
y=td['fare_amount']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.01,random_state=80)

from sklearn.linear_model import LinearRegression
lr=LinearRegression(normalize=True)
lr.fit(X_train,y_train)
print(lr.score(X_test,y_test))



In [ ]:
pred=np.round(lr.predict(ted.drop('key',axis=1)),2)
pd.read_csv('/kaggle/input/new-york-city-taxi-fare-prediction/sample_submission.csv').head()

In [ ]:
Submission=pd.DataFrame(data=pred,columns=['fare_amount'])
Submission['key']=ted['key']
Submission=Submission[['key','fare_amount']]

Submission.set_index('key',inplace=True)

Submission.to_csv('Submission.csv')

